# Train YOLOv1

In [1]:
import shutil
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('../')
from torchsummary.torchsummary import summary
from dataloader import VOC
import numpy as np
import matplotlib.pyplot as plt
import yolov1
from yolov1 import detection_collate
from yolov1 import detection_loss
from yolov1 import save_checkpoint

In [2]:
# Convolutional neural network (two convolutional layers)
class YOLOv1(nn.Module):
    def __init__(self):
        super(YOLOv1, self).__init__()
        # LAYER 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64), #, momentum=0.01
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # LAYER 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(192, momentum=0.01),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # LAYER 3
        self.layer3 = nn.Sequential(
            nn.Conv2d(192, 128, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(128, momentum=0.01),
            nn.LeakyReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # LAYER 4
        self.layer7 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer14 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer15 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer16 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # LAYER 5
        self.layer17 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer18 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())
        self.layer19 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer20 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())
        self.layer21 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())
        self.layer22 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())

        # LAYER 6
        self.layer23 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())
        self.layer24 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())

        self.fc1 = nn.Sequential(
            nn.Linear(7*7*1024, 4096),
            nn.LeakyReLU(),
            nn.Dropout(dropout_prop)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(4096, 7*7*((5)+num_classes)),
            nn.Dropout(dropout_prop),
        )


        for m in self.modules():

            if isinstance(m, nn.Conv2d):
                #nn.init.xavier_normal_(m.weight)
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity="leaky_relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.layer16(out)
        out = self.layer17(out)
        out = self.layer18(out)
        out = self.layer19(out)
        out = self.layer20(out)
        out = self.layer21(out)
        out = self.layer22(out)
        out = self.layer23(out)
        out = self.layer24(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = out.reshape((-1,7,7,((5)+num_classes)))

        return out


In [3]:
def detection_loss_test(output, target):
    # hyper parameter

    lambda_coord = 5
    lambda_noobj = 0.5

    # check batch size
    b, _, _, _ = target.shape

    # class loss
    MSE_criterion = nn.MSELoss()

    # output tensor slice
    objness1_output = output[:, :, :, 0]
    x_offset1_output = output[:, :, :, 1]
    y_offset1_output = output[:, :, :, 2]
    width_ratio1_output = output[:, :, :, 3]
    height_ratio1_output = output[:, :, :, 4]
    class_output = output[:, :, :, 5]

    # label tensor slice
    objness_label = target[:, :, :, 0]
    class_label = target[:, :, :, 1]
    x_offset_label = target[:, :, :, 2]
    y_offset_label = target[:, :, :, 3]
    width_ratio_label = target[:, :, :, 4]
    height_ratio_label = target[:, :, :, 5]
    
    noobjness_label = torch.neg(torch.add(objness_label, -1))
    #y_one_hot = one_hot(class_output, class_label).cuda()

    #print(y_one_hot)
    #print(y_one_hot.shape)
    #exit()

    obj_coord1_loss = lambda_coord * \
                      torch.sum(objness_label *
                                (torch.pow(x_offset1_output - x_offset_label, 2) +
                                 torch.pow(y_offset1_output - y_offset_label, 2)))

    obj_size1_loss = lambda_coord * \
                     torch.sum(objness_label *
                               (torch.pow(width_ratio1_output - torch.sqrt(width_ratio_label), 2) +
                                torch.pow(height_ratio1_output - torch.sqrt(height_ratio_label), 2)))

    #obj_coord2_loss = lambda_coord * \
    #                  torch.sum(objness_label *
    #                            (torch.pow(x_offset2_output - x_offset_label, 2) +
    #                             torch.pow(y_offset2_output - y_offset_label, 2)))

    #obj_size2_loss = lambda_coord * \
    #                 torch.sum(objness_label *
    #                           (torch.pow(width_ratio2_output - torch.sqrt(width_ratio_label), 2) +
    #                            torch.pow(height_ratio2_output - torch.sqrt(height_ratio_label), 2)))



    obj_class_loss = torch.sum(objness_label * torch.pow(class_output - class_label, 2))

    #noobj_class_loss = lambda_noobj * torch.sum(noobjness_label * MSE_criterion(class_output, y_one_hot))
    
    noobjness1_loss = lambda_noobj * torch.sum(noobjness_label * torch.pow(objness1_output - objness_label, 2))
    #noobjness2_loss = lambda_noobj * torch.sum(objness_label * torch.pow(objness2_output - objness_label, 2))
    
    objness1_loss = torch.sum(objness_label * torch.pow(objness1_output - objness_label, 2))
    #objness2_loss = torch.sum(objness_label * torch.pow(objness2_output - objness_label, 2))

    #total_loss = (obj_coord1_loss + obj_size1_loss + obj_coord2_loss + obj_size2_loss + noobjness1_loss + noobjness2_loss +
    #              obj_class_loss + objness1_loss + objness2_loss) / b
    total_loss = (obj_coord1_loss + obj_size1_loss + noobjness1_loss  + objness1_loss + obj_class_loss) / b

    #return total_loss, obj_coord1_loss, obj_size1_loss, obj_coord2_loss, obj_size2_loss, obj_class_loss, noobjness1_loss, noobjness2_loss, objness1_loss, objness2_loss
    return total_loss, obj_coord1_loss / b, obj_size1_loss/ b, obj_class_loss/ b, noobjness1_loss/ b, objness1_loss/ b

# 0. Vidom on

In [4]:
def create_vis_plot(_xlabel, _ylabel, _title, _legend):
    return viz.line(
        X=torch.zeros((1,)).cpu(),
        Y=torch.zeros((1, 1)).cpu(),
        opts=dict(
            xlabel=_xlabel,
            ylabel=_ylabel,
            title=_title,
            legend=_legend
        )
    )

In [5]:
def update_vis_plot(iteration, loss, window1, window2, update_type,
                    epoch_size=1):
    viz.line(
        X=torch.ones((1, 1)).cpu() * iteration,
        Y=torch.Tensor([loss]).unsqueeze(0).cpu() / epoch_size,
        win=window1,
        update=update_type
    )

In [6]:
import visdom
viz = visdom.Visdom()

vis_title = 'Yolo V1 Deepbaksu_vision (feat. martin, visionNoob) PyTorch on ' + 'VOC'
vis_legend = ['Train Loss']

iter_plot = yolov1.create_vis_plot(viz, 'Iteration', 'Total Loss', vis_title, vis_legend)

coord1_plot = yolov1.create_vis_plot(viz, 'Iteration', 'coord1', vis_title, vis_legend)
size1_plot = yolov1.create_vis_plot(viz, 'Iteration', 'size1', vis_title, vis_legend)

#coord2_plot = yolov1.create_vis_plot(viz, 'Iteration', 'coord2', vis_title, vis_legend)
#size2_plot = yolov1.create_vis_plot(viz, 'Iteration', 'size2', vis_title, vis_legend)

noobjectness1_plot = yolov1.create_vis_plot(viz, 'Iteration', 'noobjectness1', vis_title, vis_legend)
#noobjectness2_plot = yolov1.create_vis_plot(viz, 'Iteration', 'noobjectness2', vis_title, vis_legend)

objectness1_plot = yolov1.create_vis_plot(viz, 'Iteration', 'objectness1', vis_title, vis_legend)
#objectness2_plot = yolov1.create_vis_plot(viz, 'Iteration', 'objectness2', vis_title, vis_legend)

obj_cls_plot = yolov1.create_vis_plot(viz, 'Iteration', 'obj_cls', vis_title, vis_legend)
#noobj_cls_plot = yolov1.create_vis_plot(viz, 'Iteration', 'noobj_cls', vis_title, vis_legend)

# 1. Device configuration

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_epochs = 16000
num_classes = 1
batch_size = 15
learning_rate = 1e-3

dropout_prop = 0.5

# 2. Load VOC Pascal'12 Dataset

In [8]:
DATASET_PATH_MARTIN = "/media/keti-ai/AI_HARD3/DataSets/VOC_Pascal/VOC/VOCdevkit/VOC2012"
DATASET_PATH_JAEWON = "H:\VOC\VOC12\VOCdevkit_2\VOC2012"
SMALL_DATASET_PATH = "D:/dataset/person-300"
train_dataset = VOC(root = SMALL_DATASET_PATH,
                    transform=transforms.ToTensor(), cls_option = True, selective_cls="person")


VOC Parsing:   |████████████████████████████████████████| 100.0% (300/300)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (300/300)  Complete



In [9]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           collate_fn=detection_collate)

# 3. Load YOLOv1

In [10]:
net = yolov1.YOLOv1()
# visualize_weights_distribution(net)
use_cuda = torch.cuda.is_available()

In [11]:
device = torch.device("cuda" if use_cuda else "cpu")
model = YOLOv1().to(device)
#model = torch.nn.DataParallel(net, device_ids=[0]).cuda()

summary(model, (3, 448,448))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           9,472
       BatchNorm2d-2         [-1, 64, 224, 224]             128
         LeakyReLU-3         [-1, 64, 224, 224]               0
         MaxPool2d-4         [-1, 64, 112, 112]               0
            Conv2d-5        [-1, 192, 112, 112]         110,784
       BatchNorm2d-6        [-1, 192, 112, 112]             384
         LeakyReLU-7        [-1, 192, 112, 112]               0
         MaxPool2d-8          [-1, 192, 56, 56]               0
            Conv2d-9          [-1, 128, 56, 56]          24,704
      BatchNorm2d-10          [-1, 128, 56, 56]             256
        LeakyReLU-11          [-1, 128, 56, 56]               0
           Conv2d-12          [-1, 256, 56, 56]         295,168
      BatchNorm2d-13          [-1, 256, 56, 56]             512
        LeakyReLU-14          [-1, 256,

# Sanity Check for output dimention

In [12]:
images, labels = iter(train_loader).next()
images = images.to(device)
labels = labels.to(device)

In [13]:
#for just a image
num = 2
test_image = images[num]
outputs = model(torch.cuda.FloatTensor(np.expand_dims(test_image,axis=0)))
print(outputs.shape)

#for images (batch size)
outputs = model(torch.cuda.FloatTensor(images))
print(outputs.shape)

torch.Size([1, 7, 7, 6])
torch.Size([15, 7, 7, 6])


In [14]:
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-5)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

In [15]:
torch.optim.Adam?

# 4.Train the model

In [16]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):

    if (epoch == 200) or (epoch == 400) or (epoch == 600) or (epoch == 20000) or (epoch == 30000):
        scheduler.step()

    for i, (images, labels) in enumerate(train_loader):

    
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Calc Loss
        loss, \
        obj_coord1_loss, \
        obj_size1_loss, \
        obj_class_loss, \
        noobjness1_loss, \
        objness1_loss = detection_loss_test(outputs, labels)
        #objness2_loss = yolov1.detection_loss(outputs, labels)
        
        
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            
            print('Epoch ,[{}/{}] ,Step ,[{}/{}] ,lr ,{} ,total_loss ,{:.4f} ,coord1 ,{} ,size1 ,{} ,noobj_clss ,{} ,objness1 ,{} ,'
                  .format(epoch + 1,
                          num_epochs,
                          i + 1,
                          total_step,
                          [param_group['lr'] for param_group in optimizer.param_groups],
                          loss.item(),
                          obj_coord1_loss,
                          obj_size1_loss,
                          obj_class_loss,
                          noobjness1_loss,
                          objness1_loss
                          ))
            

            yolov1.update_vis_plot(viz, (epoch+1)*batch_size +(i + 1), loss.item(), iter_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_coord1_loss, coord1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_size1_loss, size1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_class_loss, obj_cls_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), noobjness1_loss, noobjectness1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), objness1_loss, objectness1_plot, None, 'append')


            

    if (epoch % 300) == 0:
        yolov1.save_checkpoint({
            'epoch': epoch + 1,
            'arch': "YOLOv1",
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, False, filename='checkpoint_{}.pth.tar'.format(epoch))


Epoch ,[1/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,1979.1355 ,coord1 ,755.8391723632812 ,size1 ,549.2973022460938 ,noobj_clss ,20.0269718170166 ,objness1 ,594.6265258789062 ,
Epoch ,[1/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,1613.6714 ,coord1 ,404.9866638183594 ,size1 ,707.9187622070312 ,noobj_clss ,32.16424560546875 ,objness1 ,465.38433837890625 ,
Epoch ,[2/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,718.2756 ,coord1 ,32.22593688964844 ,size1 ,249.67697143554688 ,noobj_clss ,3.174311876296997 ,objness1 ,414.90289306640625 ,
Epoch ,[2/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,514.2932 ,coord1 ,20.852569580078125 ,size1 ,341.1980285644531 ,noobj_clss ,7.867785453796387 ,objness1 ,141.05841064453125 ,
Epoch ,[3/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,201.2292 ,coord1 ,18.01036834716797 ,size1 ,152.3485565185547 ,noobj_clss ,1.615993857383728 ,objness1 ,26.325883865356445 ,
Epoch ,[3/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,75.4596 ,coord1 ,11.46665

Epoch ,[23/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,71.1991 ,coord1 ,3.2914113998413086 ,size1 ,60.408416748046875 ,noobj_clss ,1.9817540645599365 ,objness1 ,2.71586275100708 ,
Epoch ,[24/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,7.4588 ,coord1 ,1.1286836862564087 ,size1 ,4.270502090454102 ,noobj_clss ,0.09394919127225876 ,objness1 ,0.6845269799232483 ,
Epoch ,[24/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,15.0439 ,coord1 ,4.557737350463867 ,size1 ,7.028713226318359 ,noobj_clss ,0.08983129262924194 ,objness1 ,1.1339229345321655 ,
Epoch ,[25/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,7.4357 ,coord1 ,0.5465551614761353 ,size1 ,4.8856425285339355 ,noobj_clss ,0.09790081530809402 ,objness1 ,0.6618432998657227 ,
Epoch ,[25/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,7.2897 ,coord1 ,0.5134528279304504 ,size1 ,4.950319290161133 ,noobj_clss ,0.11415276676416397 ,objness1 ,0.4417176842689514 ,
Epoch ,[26/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,6.4777 ,coord1 ,

Epoch ,[46/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,9.3617 ,coord1 ,0.41203057765960693 ,size1 ,5.759176731109619 ,noobj_clss ,0.1579817831516266 ,objness1 ,1.5147665739059448 ,
Epoch ,[46/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,8.4075 ,coord1 ,1.9166693687438965 ,size1 ,4.0245161056518555 ,noobj_clss ,0.054206278175115585 ,objness1 ,0.586424708366394 ,
Epoch ,[47/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,6.1694 ,coord1 ,0.16406171023845673 ,size1 ,4.269114017486572 ,noobj_clss ,0.030433690175414085 ,objness1 ,0.4593166708946228 ,
Epoch ,[47/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,6.3669 ,coord1 ,0.47114259004592896 ,size1 ,4.100892543792725 ,noobj_clss ,0.011915420182049274 ,objness1 ,0.3995562493801117 ,
Epoch ,[48/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,14.9073 ,coord1 ,0.9541212320327759 ,size1 ,11.242518424987793 ,noobj_clss ,0.1422741860151291 ,objness1 ,0.6774696707725525 ,
Epoch ,[48/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,5.8546 ,co

Epoch ,[68/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,8.9839 ,coord1 ,0.6185261011123657 ,size1 ,5.972265243530273 ,noobj_clss ,0.10550404340028763 ,objness1 ,0.2914656102657318 ,
Epoch ,[69/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,8.6844 ,coord1 ,0.3859480321407318 ,size1 ,6.073273181915283 ,noobj_clss ,0.038546912372112274 ,objness1 ,0.6523514986038208 ,
Epoch ,[69/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,4.9229 ,coord1 ,0.23731447756290436 ,size1 ,2.9833872318267822 ,noobj_clss ,0.03674351051449776 ,objness1 ,0.24584485590457916 ,
Epoch ,[70/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,6.1172 ,coord1 ,0.2847512662410736 ,size1 ,4.019530296325684 ,noobj_clss ,0.01570960320532322 ,objness1 ,0.24362967908382416 ,
Epoch ,[70/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,3.9627 ,coord1 ,0.2333754152059555 ,size1 ,2.318408727645874 ,noobj_clss ,0.020125506445765495 ,objness1 ,0.2031845897436142 ,
Epoch ,[71/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,5.6500 ,co

Epoch ,[90/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,6.7397 ,coord1 ,0.4430863857269287 ,size1 ,4.336782455444336 ,noobj_clss ,0.013382304459810257 ,objness1 ,0.26023608446121216 ,
Epoch ,[91/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,4.8605 ,coord1 ,0.2058790922164917 ,size1 ,3.3050522804260254 ,noobj_clss ,0.011640271171927452 ,objness1 ,0.29263338446617126 ,
Epoch ,[91/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,6.7744 ,coord1 ,0.3496929705142975 ,size1 ,4.574100017547607 ,noobj_clss ,0.01749510131776333 ,objness1 ,0.259960800409317 ,
Epoch ,[92/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,4.2300 ,coord1 ,0.1237955242395401 ,size1 ,2.6877517700195312 ,noobj_clss ,0.020723016932606697 ,objness1 ,0.1916966438293457 ,
Epoch ,[92/16000] ,Step ,[20/20] ,lr ,[0.001] ,total_loss ,6.1552 ,coord1 ,0.5716184377670288 ,size1 ,3.5582022666931152 ,noobj_clss ,0.04468194767832756 ,objness1 ,0.270918071269989 ,
Epoch ,[93/16000] ,Step ,[10/20] ,lr ,[0.001] ,total_loss ,5.7541 ,co

...

Epoch ,[2654/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,3.0529 ,coord1 ,0.06275735795497894 ,size1 ,1.8567078113555908 ,noobj_clss ,0.0002554146049078554 ,objness1 ,0.03564281016588211 ,
Epoch ,[2655/16000] ,Step ,[10/20] ,lr ,[0.0009025] ,total_loss ,3.4232 ,coord1 ,0.06702031195163727 ,size1 ,2.4420745372772217 ,noobj_clss ,0.00047413568245247006 ,objness1 ,0.030353643000125885 ,
Epoch ,[2655/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,5.2924 ,coord1 ,0.050869788974523544 ,size1 ,4.40718936920166 ,noobj_clss ,0.0004986205603927374 ,objness1 ,0.0260822381824255 ,
Epoch ,[2656/16000] ,Step ,[10/20] ,lr ,[0.0009025] ,total_loss ,3.0456 ,coord1 ,0.06933347880840302 ,size1 ,2.0515024662017822 ,noobj_clss ,0.00023252717801369727 ,objness1 ,0.024972565472126007 ,
Epoch ,[2656/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,3.8438 ,coord1 ,0.07105602324008942 ,size1 ,2.620831251144409 ,noobj_clss ,0.0003133118443656713 ,objness1 ,0.026170512661337852 ,
Epoch ,[2657/16000

Epoch ,[2675/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,4.2979 ,coord1 ,0.03865976631641388 ,size1 ,3.3918652534484863 ,noobj_clss ,0.0001576774229761213 ,objness1 ,0.034334342926740646 ,
Epoch ,[2676/16000] ,Step ,[10/20] ,lr ,[0.0009025] ,total_loss ,4.1473 ,coord1 ,0.0649743378162384 ,size1 ,3.2297134399414062 ,noobj_clss ,0.000230032776016742 ,objness1 ,0.027269288897514343 ,
Epoch ,[2676/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,4.8487 ,coord1 ,0.05249504745006561 ,size1 ,3.740226984024048 ,noobj_clss ,0.0004016966267954558 ,objness1 ,0.015874002128839493 ,
Epoch ,[2677/16000] ,Step ,[10/20] ,lr ,[0.0009025] ,total_loss ,5.5202 ,coord1 ,0.06161404028534889 ,size1 ,4.321191310882568 ,noobj_clss ,0.0002889609313569963 ,objness1 ,0.027097079902887344 ,
Epoch ,[2677/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,5.6878 ,coord1 ,0.09174081683158875 ,size1 ,4.271949291229248 ,noobj_clss ,0.0002487848396413028 ,objness1 ,0.030185114592313766 ,
Epoch ,[2678/16000] 

Epoch ,[2696/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,3.8136 ,coord1 ,0.07470758259296417 ,size1 ,2.7716903686523438 ,noobj_clss ,9.676607442088425e-05 ,objness1 ,0.027631761506199837 ,
Epoch ,[2697/16000] ,Step ,[10/20] ,lr ,[0.0009025] ,total_loss ,3.9866 ,coord1 ,0.0771288201212883 ,size1 ,2.686110496520996 ,noobj_clss ,0.00012096003047190607 ,objness1 ,0.03097766824066639 ,
Epoch ,[2697/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,4.7188 ,coord1 ,0.07473313063383102 ,size1 ,3.4882750511169434 ,noobj_clss ,0.00045856626820750535 ,objness1 ,0.035531919449567795 ,
Epoch ,[2698/16000] ,Step ,[10/20] ,lr ,[0.0009025] ,total_loss ,4.4522 ,coord1 ,0.07096559554338455 ,size1 ,3.3011929988861084 ,noobj_clss ,0.0001100801455322653 ,objness1 ,0.026073826476931572 ,
Epoch ,[2698/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,3.9267 ,coord1 ,0.0459110364317894 ,size1 ,2.9055659770965576 ,noobj_clss ,0.00015456698019988835 ,objness1 ,0.023025384172797203 ,
Epoch ,[2699/160

Epoch ,[2717/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,5.5887 ,coord1 ,0.055554408580064774 ,size1 ,4.208277225494385 ,noobj_clss ,0.0006054640980437398 ,objness1 ,0.0202019065618515 ,
Epoch ,[2718/16000] ,Step ,[10/20] ,lr ,[0.0009025] ,total_loss ,3.7924 ,coord1 ,0.04383763298392296 ,size1 ,2.8819031715393066 ,noobj_clss ,0.00019133486784994602 ,objness1 ,0.031236890703439713 ,
Epoch ,[2718/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,5.2102 ,coord1 ,0.05602351203560829 ,size1 ,4.119403839111328 ,noobj_clss ,0.00042852695332840085 ,objness1 ,0.02221689000725746 ,
Epoch ,[2719/16000] ,Step ,[10/20] ,lr ,[0.0009025] ,total_loss ,5.4625 ,coord1 ,0.050439950078725815 ,size1 ,4.125802040100098 ,noobj_clss ,0.00047928772983141243 ,objness1 ,0.0214003287255764 ,
Epoch ,[2719/16000] ,Step ,[20/20] ,lr ,[0.0009025] ,total_loss ,3.4228 ,coord1 ,0.06428490579128265 ,size1 ,2.4042794704437256 ,noobj_clss ,0.00046964551438577473 ,objness1 ,0.034864988178014755 ,
Epoch ,[2720/1600

KeyboardInterrupt: 

In [ ]:
labels[2]

In [ ]:
outputs = model(images)

In [ ]:
class_output = outputs[:, :, :, 10:]
class_label = labels[:, :, :, 1]
y_one_hot = one_hot(class_output, class_label)
y_one_hot[0,:,:,0]

In [ ]:
print(class_output[0])
print(class_label[0])

In [ ]:
class_label[0]

In [ ]:
class_label/shape

In [ ]:
objness_label = labels[:, :, :, 0]
noobjness_label = torch.neg(torch.add(objness_label, -1))

In [ ]:
labels.shape

In [ ]:
noobjness_label[0]

In [ ]:
labels[0,:,:,0]

In [ ]:
a = 

In [ ]:
outputs = model(images)

In [ ]:
# Calc Loss
loss, \
obj_coord1_loss, \
obj_size1_loss, \
obj_class_loss, \
noobj_class_loss, \
objness1_loss = detection_loss_test(outputs, labels)
    

In [ ]:
print("total loss:\t\t",loss)
print("obj_coord1_loss:\t",obj_coord1_loss)
print("obj_size1_loss:\t\t",obj_size1_loss)
print("obj_class_loss:\t\t",obj_class_loss)
print("noobj_class_loss:\t",noobj_class_loss)
print("objness1_loss:\t\t",objness1_loss)

In [ ]:
width_ratio1_output = outputs[:, :, :, 3]

In [ ]:
a = torch.sqrt(width_ratio1_output)

In [ ]:
print(width_ratio1_output)

# TEST

In [ ]:
# Test the model

test_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           collate_fn=detection_collate)

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
predicted.shape

In [ ]:
labels.shape

In [ ]:
outputs[0].shape

In [ ]:
images.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline 

In [ ]:
outputs[0].shape

In [ ]:
outputs[0][0][0]

In [ ]:
print(outputs[0, :, :, 0])
print(outputs[0, :, :, 5])

In [ ]:
print(outputs[0,0,0,:])
print(outputs[1,0,0,:])
print(outputs[2,0,0,:])

In [ ]:
plt.imshow(np.transpose(images[num],(1,2,0)))
plt.show()
plt.imshow(outputs.detach().cpu().numpy()[0,:, :, 0])
plt.show()
plt.imshow(outputs.detach().cpu().numpy()[0,:, :, 5])
plt.show()

In [ ]:
image.shape

In [ ]:
np.expand_dims(test_image,axis=0).shape